#### Indian sign laguages (CNN + RNN)

In [1]:
''' keras video genrator for data augmentation of the video'''
!pip install keras-video-generators
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


In [2]:
%cd /content/drive/My Drive/PROJECT_DATA

/content/drive/My Drive/PROJECT_DATA


In [3]:
'''importing libraries'''
import os
import sys
import glob
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from keras_video import VideoFrameGenerator

Using TensorFlow backend.


In [4]:
from keras_video import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('Hands_only_OpenPose/*')]
classes.sort()
# some global params
SIZE = (150, 150)
CHANNELS = 3
NBFRAME = 11
BS = 8
# pattern to get videos and classes
glob_pattern='Hands_only_OpenPose/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.7)
    # horizontal_flip=True,
    # rotation_range=8,
    # width_shift_range=.2,
    # height_shift/_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.3, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class friday, validation count: 30, train count: 72
class monday, validation count: 34, train count: 81
class saturday, validation count: 30, train count: 73
class sunday, validation count: 38, train count: 90
class thursday, validation count: 30, train count: 72
class tuesday, validation count: 39, train count: 93
class wednesday, validation count: 31, train count: 75
Total data: 7 classes for 556 files for train


In [5]:
valid = train.get_validation_generator()

Total data: 7 classes for 232 files for validation


In [0]:
# import keras_video.utils
# keras_video.utils.show_sample(train)

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical

In [28]:
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(11, 150,150,3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(7, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_4 (ConvLSTM2D)  (None, 150, 148, 64)      51712     
_________________________________________________________________
dropout_4 (Dropout)          (None, 150, 148, 64)      0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1420800)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               142080100 
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 707       
Total params: 142,132,519
Trainable params: 142,132,519
Non-trainable params: 0
_________________________________________________________________


In [0]:
# from sklearn.utils import class_weight
# def get_weight(y):
#     class_weight_current =  class_weight.compute_class_weight('balanced', np.unique(y), y)
#     return class_weight_current

In [0]:
# class_weight = get_weight(train.classes)
# class_weight

In [0]:
from keras.optimizers import Adam
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping
filepath="/content/drive/My Drive/Models/openpose.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True, verbose=1)
reduce_on_plateau = ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.1, patience=4, verbose=1)
early_stopping = EarlyStopping( monitor='val_loss',patience=10,verbose=1,restore_best_weights=True)
callbacks_list = [checkpoint, reduce_on_plateau, early_stopping]

In [33]:
history = model.fit_generator(train, epochs=20,
                              callbacks=callbacks_list, verbose = 1,
                              validation_data = valid,
                              workers=5)

Epoch 1/20
69/69 [==============================] - 109s 2s/step - loss: 1.8832 - acc: 0.2536 - val_loss: 1.9223 - val_acc: 0.2802

Epoch 00001: val_loss improved from inf to 1.92226, saving model to /content/drive/My Drive/Models/openpose.hdf5
Epoch 2/20
69/69 [==============================] - 110s 2s/step - loss: 1.7305 - acc: 0.3279 - val_loss: 1.6510 - val_acc: 0.3922

Epoch 00002: val_loss improved from 1.92226 to 1.65099, saving model to /content/drive/My Drive/Models/openpose.hdf5
Epoch 3/20
69/69 [==============================] - 96s 1s/step - loss: 1.6602 - acc: 0.3641 - val_loss: 1.6711 - val_acc: 0.3621

Epoch 00003: val_loss did not improve from 1.65099
Epoch 4/20
69/69 [==============================] - 97s 1s/step - loss: 1.5883 - acc: 0.4022 - val_loss: 1.5474 - val_acc: 0.4741

Epoch 00004: val_loss improved from 1.65099 to 1.54739, saving model to /content/drive/My Drive/Models/openpose.hdf5
Epoch 5/20
69/69 [==============================] - 104s 2s/step - loss: 1.4